In [25]:
import pandas as pd
import numpy as np
import math

### Pull data

In [101]:
in_path = 'data/winemag-data-geocoded.csv'
df = pd.read_csv(in_path)

red_white_path = 'data/wine_varietal.csv'
red_white_label = pd.read_csv(red_white_path)

### Label red/white wines

In [102]:
df.groupby('variety').size().sort_values(ascending=False).head()

red = list(red_white_label[red_white_label.Type=='Red'].Name.str.strip().unique()) + ['Verduzzo Friulano ','Tintilia ']
white = list(red_white_label[red_white_label.Type=='White'].Name.str.strip().unique())

In [103]:
df.ix[df.variety.isin(red),'color'] = 'red'
df.ix[df.variety.isin(white),'color'] = 'white'
df.ix[df.color.isna(),'color'] = 'missing'
df.color.unique()

C:\Users\geoffc.REDMOND\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


array(['white', 'red'], dtype=object)

In [104]:
df.groupby(['color']).size().sort_values(ascending=False).to_frame()

,0
color,
red,13755
white,5785


### Clean-up province mappings (WIP)

In [105]:
#clean-up province mappings
print(df.groupby('province')['lat'].mean())
print(df.groupby('province')['long'].mean())

province
Central Italy         43.176554
Italy Other           43.393173
Lombardy              45.538123
Northeastern Italy    46.088512
Northwestern Italy    44.793867
Piedmont              44.687971
Sicily & Sardinia     38.024212
Southern Italy        41.083497
Tuscany               43.289044
Veneto                45.534886
Name: lat, dtype: float64
province
Central Italy         12.746926
Italy Other           11.205732
Lombardy              10.295514
Northeastern Italy    12.232828
Northwestern Italy     8.255510
Piedmont               8.173752
Sicily & Sardinia     13.358211
Southern Italy        15.348579
Tuscany               11.330808
Veneto                11.528472
Name: long, dtype: float64


### Find number of wineries within 20 miles

In [106]:
temp = df[~df.long.isna()]
temp = temp.groupby('winery')[['long','lat']].mean().reset_index()
temp.head()

,winery,long,lat
0,460 Casina Bric,7.928460,44.618212
1,Abbadia Ardenga,11.544132,43.087302
2,Abbazia Monte Oliveto,11.544765,43.175771
3,Abbazia Santa Anastasia,14.084965,37.972501
4,Abbazia di Novacella,11.648464,46.742693


In [49]:
#function computes the distance between two lat/lon
#source: https://community.esri.com/groups/coordinate-reference-systems/blog/2017/10/05/haversine-formula
def geo_dist(coord_1,coord_2):
    # Coordinates in decimal degrees (e.g. 2.89078, 12.79797)
    lon1, lat1 = coord_1
    lon2, lat2 = coord_2

    R = 6371000  # radius of Earth in meters
    phi_1 = math.radians(lat1)
    phi_2 = math.radians(lat2)

    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi_1) * math.cos(phi_2) * math.sin(delta_lambda / 2.0) ** 2
    
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    km = R * c/1000  # output distance in km
    return km

In [50]:
[temp.lat[0],temp.long[0]]

[44.618212400000004, 7.9284596999999986]

In [56]:
length = temp.winery.size
cols = ['winery_1','winery_2','dist']
lst = []
for i in range(0,length):
    for j in range(0,int(length)):
        if i < j:
            lst.append([temp.winery[i],temp.winery[j],geo_dist([temp.lat[i],temp.long[i]],[temp.lat[j],temp.long[j]])])
            if i % 100 == 0 and j % 100 == 0:
                print(i,j)

temp_2 = pd.DataFrame(lst,columns=cols)

0 100
0 200
0 300
0 400
0 500
0 600
0 700
0 800
0 900
0 1000
0 1100
0 1200
0 1300
0 1400
0 1500
0 1600
0 1700
0 1800
0 1900
0 2000
0 2100
0 2200
0 2300
0 2400
0 2500
100 0
100 200
100 300
100 400
100 500
100 600
100 700
100 800
100 900
100 1000
100 1100
100 1200
100 1300
100 1400
100 1500
100 1600
100 1700
100 1800
100 1900
100 2000
100 2100
100 2200
100 2300
100 2400
100 2500
200 0
200 100
200 300
200 400
200 500
200 600
200 700
200 800
200 900
200 1000
200 1100
200 1200
200 1300
200 1400
200 1500
200 1600
200 1700
200 1800
200 1900
200 2000
200 2100
200 2200
200 2300
200 2400
200 2500
300 0
300 100
300 200
300 400
300 500
300 600
300 700
300 800
300 900
300 1000
300 1100
300 1200
300 1300
300 1400
300 1500
300 1600
300 1700
300 1800
300 1900
300 2000
300 2100
300 2200
300 2300
300 2400
300 2500
400 0
400 100
400 200
400 300
400 500
400 600
400 700
400 800
400 900
400 1000
400 1100
400 1200
400 1300
400 1400
400 1500
400 1600
400 1700
400 1800
400 1900
400 2000
400 2100
400 2200
400 2

In [ ]:
#export results
out_path = 'data/winery-distances.csv'
temp_2.to_csv(out_path)

In [107]:
#find count within dist_cutoff km
dist_cutoff = 5

dist_table = temp_2[temp_2['dist']<5].groupby('winery_1').size().reset_index()
dist_table['CountWithin5Km'] = dist_table[0]
dist_table = dist_table.drop(columns=0)

In [108]:
dist_table.head()

,winery_1,CountWithin5Km
0,460 Casina Bric,98
1,Abbadia Ardenga,20
2,Abbazia Monte Oliveto,1
3,Abbazia di Novacella,1
4,Abbona Marziano,32


In [110]:
#add dist to main frame
df = df.merge(right=dist_table,how='left',left_on='winery',right_on='winery_1').drop(columns='winery_1')
df.ix[df.CountWithin5Km.isna(),'CountWithin5Km'] = 0.0
df[df['winery']=='460 Casina Bric']

C:\Users\geoffc.REDMOND\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


,Unnamed: 0,Unnamed: 0.1,country,description,designation,points,price,province,region_1,region_2,...,taster_twitter_handle,title,variety,winery,name,lat,long,address,color,CountWithin5Km
2164,2164,13851,Italy,"Aromas of chocolate, mocha and raisin lead the...",NaN,86,45.0,Piedmont,Barolo,NaN,...,@kerinokeefe,460 Casina Bric 2011 Barolo,Nebbiolo,460 Casina Bric,460 Casina Bric,44.618212,7.92846,"Via Sorello, Cascina Bricco Fraz. Vergne 12060...",red,98.0
3176,3176,20886,Italy,Closed cellar aromas burn off to reveal black ...,Bricco delle Viole,88,60.0,Piedmont,Barolo,NaN,...,@kerinokeefe,460 Casina Bric 2010 Bricco delle Viole (Barolo),Nebbiolo,460 Casina Bric,460 Casina Bric,44.618212,7.92846,"Via Sorello, Cascina Bricco Fraz. Vergne 12060...",red,98.0
4438,4438,29123,Italy,This offers a classic Nebbiolo nose of rose pe...,Bricco delle Viole,89,65.0,Piedmont,Barolo,NaN,...,@kerinokeefe,460 Casina Bric 2011 Bricco delle Viole (Barolo),Nebbiolo,460 Casina Bric,460 Casina Bric,44.618212,7.92846,"Via Sorello, Cascina Bricco Fraz. Vergne 12060...",red,98.0


In [111]:
df_out = df
out_path = 'data/winemag-data-geocoded-clean.csv'
df_out.to_csv(out_path)